In [ ]:
import cv2
import numpy as np
import os
import glob
import math
import matplotlib.pyplot as plt
import time

INPUT_IMAGE_DIR = 'images/4 Cropped_Images/crop_001_20150119_021648.png'
OUTPUT_IMAGE_DIR = 'images/oriented'
REFERENCE_MAP_PATH = 'T2.jpg'

ROTATION_STEP = 1
PROFILE_LENGTH = 256
SHOW_PLOTS = True

os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

print("--- Configuration Summary (v4.6 Shaded Flux Analysis) ---")
print(f"Input path/pattern: {INPUT_IMAGE_DIR}")
print(f"Output directory:   {os.path.abspath(OUTPUT_IMAGE_DIR)}")
print(f"Reference Map:      {REFERENCE_MAP_PATH}")
print(f"Rotation Step:      {ROTATION_STEP} degrees")
print(f"Show Plots:         {SHOW_PLOTS}")
print("-" * 40)

if not os.path.exists(REFERENCE_MAP_PATH):
    print(f"\n!!! CRITICAL WARNING: Reference map '{REFERENCE_MAP_PATH}' not found. !!!\n")

def segment_planet_and_get_properties(image_color):
    if image_color is None: return None, None, None, None
    
    img_for_segmentation = image_color.copy()
    if len(img_for_segmentation.shape) > 2 and img_for_segmentation.shape[2] == 3:
        gray = cv2.cvtColor(img_for_segmentation, cv2.COLOR_BGR2GRAY)
    elif len(img_for_segmentation.shape) == 2:
        gray = img_for_segmentation
    else:
        print("  Error: Invalid image format for segmentation.")
        return None, None, None, None
    
    height, width = gray.shape
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    block_size = int(min(height, width) / 15) 
    if block_size % 2 == 0: block_size += 1
    if block_size <= 1: block_size = 11 
    C_value = 2 
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, block_size, C_value)

    if np.mean(thresh[blurred > np.mean(blurred)]) < 127 and np.mean(blurred) > 30:
         print("  Inverting threshold mask.")
         thresh = cv2.bitwise_not(thresh)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    thresh_cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh_cleaned = cv2.morphologyEx(thresh_cleaned, cv2.MORPH_OPEN, kernel, iterations=2)
    
    contours, _ = cv2.findContours(thresh_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        
        if area > 0.10 * width * height:
            planet_mask_out = np.zeros(gray.shape, dtype="uint8")
            cv2.drawContours(planet_mask_out, [largest_contour], -1, 255, thickness=cv2.FILLED)
            
            ((cx, cy), radius) = cv2.minEnclosingCircle(largest_contour)
            print(f"  Planet segmented: Approx. Center=({int(cx)}, {int(cy)}), Radius={int(radius)}")
            return planet_mask_out, int(cx), int(cy), int(radius)
        else:
            print(f"  Largest contour area ({area:.0f}) too small for planet. Segmentation failed.")
    else:
        print("  No contours found for segmentation.")
    return None, None, None, None

def rotate_image_same_canvas(image, angle, center):
    if image is None: return None
    if center is None or center[0] is None or center[1] is None:
        return image 
    (h, w) = image.shape[:2]
    try:
        center_tuple = (float(center[0]), float(center[1]))
        M = cv2.getRotationMatrix2D(center_tuple, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        return rotated
    except Exception as e:
        print(f"  Error during same-canvas rotation: {e}")
        return None

def rotate_image_expanded_canvas_robust(image, angle_degrees, center_xy_original):
    if image is None: return None, center_xy_original
    if center_xy_original is None or \
       center_xy_original[0] is None or center_xy_original[1] is None:
        return image, center_xy_original
    (h_orig, w_orig) = image.shape[:2]
    center_x_orig_float, center_y_orig_float = float(center_xy_original[0]), float(center_xy_original[1])
    M = cv2.getRotationMatrix2D((center_x_orig_float, center_y_orig_float), float(angle_degrees), 1.0)
    corners_orig = np.array([[0, 0], [w_orig - 1, 0], [w_orig - 1, h_orig - 1], [0, h_orig - 1]], dtype="float32")
    ones = np.ones(shape=(len(corners_orig), 1)); points_ones = np.hstack([corners_orig, ones])
    transformed_corners = M @ points_ones.T; transformed_corners = transformed_corners.T
    x_coords, y_coords = transformed_corners[:, 0], transformed_corners[:, 1]
    x_min, x_max = np.min(x_coords), np.max(x_coords); y_min, y_max = np.min(y_coords), np.max(y_coords)
    new_w = math.ceil(x_max - x_min); new_h = math.ceil(y_max - y_min)
    M[0, 2] -= x_min; M[1, 2] -= y_min
    try:
        rotated_image = cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        original_center_homogeneous = np.array([[center_x_orig_float], [center_y_orig_float], [1.0]])
        new_center_transformed = M @ original_center_homogeneous
        new_center_in_expanded = (int(round(new_center_transformed[0,0])), int(round(new_center_transformed[1,0])))
        return rotated_image, new_center_in_expanded
    except Exception as e:
        print(f"  Error during robust expanded canvas rotation: {e}")
        return image, (int(round(center_x_orig_float)), int(round(center_y_orig_float)))

def calculate_band_orientation_metric(image_for_metric, disk_center_in_image, disk_radius):
    if image_for_metric is None or image_for_metric.size == 0: return float('inf')
    if disk_center_in_image is None or disk_radius is None or disk_radius <=0: return float('inf')
    if len(image_for_metric.shape) > 2: gray_image = cv2.cvtColor(image_for_metric, cv2.COLOR_BGR2GRAY)
    else: gray_image = image_for_metric.copy()
    mask = np.zeros(gray_image.shape[:2], dtype="uint8")
    center_pt = (int(round(disk_center_in_image[0])), int(round(disk_center_in_image[1])))
    radius_val = int(round(disk_radius))
    cv2.circle(mask, center_pt, radius_val, 255, -1)
    if np.sum(mask) < 100 : return float('inf')
    masked_gray_image = cv2.bitwise_and(gray_image, gray_image, mask=mask)
    true_points = np.argwhere(mask)
    if true_points.shape[0] == 0: return float('inf')
    top_left, bottom_right = true_points.min(axis=0), true_points.max(axis=0)
    active_region = masked_gray_image[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    if active_region.shape[0] < 2 or active_region.shape[1] < 2: return float('inf')
    diff_x = np.abs(active_region[:, :-1].astype(np.float32) - active_region[:, 1:].astype(np.float32))
    mask_for_diff = mask[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    valid_diff_mask = np.logical_and(mask_for_diff[:, :-1] > 0, mask_for_diff[:, 1:] > 0)
    sum_diff, num_valid_pairs = np.sum(diff_x[valid_diff_mask]), np.sum(valid_diff_mask)
    return (sum_diff / num_valid_pairs) if num_valid_pairs > 0 else float('inf')

def calculate_image_flux(image_data, mask=None):
    if image_data is None: return 0.0
    if len(image_data.shape) == 3 and image_data.shape[2] == 3:
        gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    elif len(image_data.shape) == 2:
        gray = image_data
    else:
        return 0.0
    if mask is not None:
        if mask.shape != gray.shape:
            return np.sum(gray.astype(np.float64))
        return np.sum(gray[mask == 255].astype(np.float64))
    else:
        return np.sum(gray.astype(np.float64))

def get_reference_profile(map_image_path, profile_length=256):
    map_img = cv2.imread(map_image_path, cv2.IMREAD_COLOR)
    if map_img is None: return None
    map_gray = cv2.cvtColor(map_img, cv2.COLOR_BGR2GRAY); profile = np.mean(map_gray, axis=1)
    original_indices = np.linspace(0, 1, len(profile)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, profile); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def get_vertical_profile(image, cx, cy, radius, profile_length=256):
    if image is None or cx is None or cy is None or radius is None or radius <= 0: return None
    if len(image.shape) > 2: gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: gray = image.copy()
    h, w_img = image.shape[:2]; y_start = max(0, int(round(cy - radius))); y_end = min(h, int(round(cy + radius)))
    cx_int = int(round(cx));
    if not (0 <= cx_int < w_img) or y_start >= y_end: return None
    vertical_slice = gray[y_start:y_end, cx_int]
    if vertical_slice.size == 0: return None
    original_indices = np.linspace(0, 1, len(vertical_slice)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, vertical_slice); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def compare_profiles_xcorr(profile1, profile_ref):
    if profile1 is None or profile_ref is None or len(profile1) != len(profile_ref): return 0.0
    p1, pref = np.asarray(profile1), np.asarray(profile_ref); correlation = np.sum(p1 * pref) / len(p1)
    return np.clip(correlation, -1.0, 1.0)

reference_profile = get_reference_profile(REFERENCE_MAP_PATH, profile_length=PROFILE_LENGTH)
if reference_profile is None:
    print("\n!!! WARNING: Could not load reference profile. !!!\n")

print("\n--- Starting Image Processing (v4.6 Shaded Flux Analysis) ---")
input_files = sorted(glob.glob(INPUT_IMAGE_DIR))

if not input_files:
    print(f"Error: No images found: '{INPUT_IMAGE_DIR}'")
else:
    print(f"Found {len(input_files)} images to process...")

total_start_time = time.time()
processed_count = 0

for img_path in input_files:
    img_proc_start_time = time.time()
    filename = os.path.basename(img_path)
    base_output_name = os.path.splitext(filename)[0]
    output_filename_chosen_rotated = base_output_name + '_oriented.png'
    output_path_chosen_rotated = os.path.join(OUTPUT_IMAGE_DIR, output_filename_chosen_rotated)

    print(f"\nProcessing: {filename}")
    img_original = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img_original is None:
        print(f"  Error loading {img_path}. Skipping.")
        continue

    planet_mask, cx_seg, cy_seg, radius_seg = segment_planet_and_get_properties(img_original)
    
    if planet_mask is None or cx_seg is None:
        print(f"  Skipping {filename} - Could not segment planet robustly.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_SEGMENTATION_FAILED.png'), img_original)
        continue
    
    original_pivot_center = (cx_seg, cy_seg) 
    radius_for_analysis = radius_seg

    flux_original_total = calculate_image_flux(img_original)
    print(f"  Flux 1 (Original Total): {flux_original_total:,.0f}")

    flux_masked_on_original = calculate_image_flux(img_original, mask=planet_mask)
    print(f"  Flux 2 (Original Masked Area, R~{radius_for_analysis}): {flux_masked_on_original:,.0f}")

    img_original_with_shading_display = img_original.copy()
    overlay_color_display = np.zeros_like(img_original_with_shading_display)
    overlay_color_display[planet_mask == 255] = [0, 100, 0] 
    alpha_display = 0.4 
    cv2.addWeighted(overlay_color_display, alpha_display, img_original_with_shading_display, 1 - alpha_display, 0, img_original_with_shading_display)
    # Optionally draw contour of the mask for clarity
    contours_for_draw, _ = cv2.findContours(planet_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_for_draw:
        cv2.drawContours(img_original_with_shading_display, contours_for_draw, -1, (0,255,0), 1)


    best_angle_metric = 0
    min_metric = float('inf')
    print(f"  Analyzing rotations for band metric (Center: {original_pivot_center}, R: {radius_for_analysis})...")
    
    for angle_deg_test in range(0, 360, ROTATION_STEP):
        temp_rotated_same_canvas = rotate_image_same_canvas(img_original, angle_deg_test, original_pivot_center)
        if temp_rotated_same_canvas is None: continue
        metric = calculate_band_orientation_metric(temp_rotated_same_canvas, original_pivot_center, radius_for_analysis)
        if metric < min_metric:
            min_metric = metric
            best_angle_metric = angle_deg_test
            
    if min_metric == float('inf'):
        print(f"  Failed to find optimal rotation for {filename} via metric. Saving original.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_METRIC_FAILED.png'), img_original)
        continue
    print(f"  Initial metric-based best angle: {best_angle_metric}° (Metric: {min_metric:.3f})")

    angle_opt1 = best_angle_metric
    angle_opt2 = (best_angle_metric + 180) % 360

    img_expanded_rotated_opt1, center_in_opt1_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt1, original_pivot_center)
    img_expanded_rotated_opt2, center_in_opt2_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt2, original_pivot_center)

    if img_expanded_rotated_opt1 is None or img_expanded_rotated_opt2 is None:
        print(f"  Error during final full-view rotations for {filename}. Saving original.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_FINAL_ROT_FAILED.png'), img_original)
        continue

    print("  --- Profile Comparison Diagnostic ---")
    profile_suggestion = "Profile Check Failed or Ref Missing"
    chosen_angle_final = angle_opt1 

    if reference_profile is not None:
        profile1 = get_vertical_profile(img_expanded_rotated_opt1, center_in_opt1_canvas[0], center_in_opt1_canvas[1], radius_for_analysis, PROFILE_LENGTH)
        profile2 = get_vertical_profile(img_expanded_rotated_opt2, center_in_opt2_canvas[0], center_in_opt2_canvas[1], radius_for_analysis, PROFILE_LENGTH)

        if profile1 is not None and profile2 is not None:
            score1 = compare_profiles_xcorr(profile1, reference_profile)
            score2 = compare_profiles_xcorr(profile2, reference_profile)
            print(f"    Profile Score: Angle {angle_opt1}° (Opt1) = {score1:.4f}")
            print(f"    Profile Score: Angle {angle_opt2}° (Opt2) = {score2:.4f}")
            if score1 >= score2: chosen_angle_final = angle_opt1; profile_suggestion = f"Suggests Angle {angle_opt1}° (Score: {score1:.4f})"
            else: chosen_angle_final = angle_opt2; profile_suggestion = f"Suggests Angle {angle_opt2}° (Score: {score2:.4f})"
        else:
            print("    Could not extract profiles for comparison. Using metric angle.")
            profile_suggestion = f"Profile Extraction Failed. Using metric Angle {chosen_angle_final}°"
    else:
        print("    Reference profile not available. Using metric-based angle.")
        profile_suggestion = f"No Ref Profile. Using metric Angle {chosen_angle_final}°"
        
    print(f"    {profile_suggestion}")
    print(f"  Final Chosen Angle for {filename}: {chosen_angle_final} degrees")

    img_chosen_full_rotated_expanded_display, center_in_chosen_expanded_canvas = \
        rotate_image_expanded_canvas_robust(img_original, chosen_angle_final, original_pivot_center)
    
    if img_chosen_full_rotated_expanded_display is None:
        print(f"  ERROR: Final chosen rotation for display failed for {filename}.")
        img_chosen_full_rotated_expanded_display = img_original 
        flux_rotated_total = flux_original_total 
        print("         Displaying original image instead for Panel 3.")
    else:
        flux_rotated_total = calculate_image_flux(img_chosen_full_rotated_expanded_display)

    print(f"  Flux 3 (Rotated Total, Expanded Canvas): {flux_rotated_total:,.0f}")
    flux_diff = flux_original_total - flux_rotated_total
    flux_diff_percent = (flux_diff / flux_original_total * 100) if flux_original_total != 0 else 0
    print(f"  Flux Difference (Original - Rotated_Expanded): {flux_diff:,.0f} ({flux_diff_percent:.2f}%)")

    print(f"  Saving Chosen Full-View Orientation (Angle {chosen_angle_final} deg) to: {output_path_chosen_rotated}")
    save_success = cv2.imwrite(output_path_chosen_rotated, img_chosen_full_rotated_expanded_display)
    if not save_success: print(f"  Warning: Failed to save {output_path_chosen_rotated}.")

    if SHOW_PLOTS:
        fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 

        axes[0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"1. Original: {filename}\nFlux: {flux_original_total:,.0f}", fontsize=10)
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(img_original_with_shading_display, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"2. Original + Planet Shaded\nFlux (Masked): {flux_masked_on_original:,.0f}", fontsize=10)
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(img_chosen_full_rotated_expanded_display, cv2.COLOR_BGR2RGB))
        axes[2].set_title(f"3. Full Rotated (Angle {chosen_angle_final}°)\nFlux: {flux_rotated_total:,.0f}\n{profile_suggestion}", fontsize=10)
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()

    processed_count += 1
    img_proc_elapsed_time = time.time() - img_proc_start_time
    print(f"  Time taken for {filename}: {img_proc_elapsed_time:.2f} seconds.")

total_elapsed_time = time.time() - total_start_time
print("\n--- Processing finished ---")
if processed_count > 0:
    print(f"Successfully processed {processed_count} out of {len(input_files)} images.")
    print(f"Total time: {total_elapsed_time:.2f} seconds.")
    print(f"Average time per image: {total_elapsed_time / processed_count:.2f} seconds.")
else:
    print(f"No images were successfully processed from {len(input_files)} found.")
print(f"Output saved in: {os.path.abspath(OUTPUT_IMAGE_DIR)}")


In [ ]:
import cv2
import numpy as np
import os
import glob
import math
import matplotlib.pyplot as plt
import time

INPUT_IMAGE_DIR = 'images/4 Cropped_Images/*.png'
OUTPUT_IMAGE_DIR = 'images/oriented'
REFERENCE_MAP_PATH = 'T2.jpg'

ROTATION_STEP = 1
PROFILE_LENGTH = 256
SHOW_PLOTS = True

os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

print("--- Configuration Summary (v4.6 Shaded Flux Analysis) ---")
print(f"Input path/pattern: {INPUT_IMAGE_DIR}")
print(f"Output directory:   {os.path.abspath(OUTPUT_IMAGE_DIR)}")
print(f"Reference Map:      {REFERENCE_MAP_PATH}")
print(f"Rotation Step:      {ROTATION_STEP} degrees")
print(f"Show Plots:         {SHOW_PLOTS}")
print("-" * 40)

if not os.path.exists(REFERENCE_MAP_PATH):
    print(f"\n!!! CRITICAL WARNING: Reference map '{REFERENCE_MAP_PATH}' not found. !!!\n")

def segment_planet_and_get_properties(image_color):
    if image_color is None: return None, None, None, None
    
    img_for_segmentation = image_color.copy()
    if len(img_for_segmentation.shape) > 2 and img_for_segmentation.shape[2] == 3:
        gray = cv2.cvtColor(img_for_segmentation, cv2.COLOR_BGR2GRAY)
    elif len(img_for_segmentation.shape) == 2:
        gray = img_for_segmentation
    else:
        print("  Error: Invalid image format for segmentation.")
        return None, None, None, None
    
    height, width = gray.shape
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    block_size = int(min(height, width) / 15) 
    if block_size % 2 == 0: block_size += 1
    if block_size <= 1: block_size = 11 
    C_value = 2 
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, block_size, C_value)

    if np.mean(thresh[blurred > np.mean(blurred)]) < 127 and np.mean(blurred) > 30:
         print("  Inverting threshold mask.")
         thresh = cv2.bitwise_not(thresh)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    thresh_cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh_cleaned = cv2.morphologyEx(thresh_cleaned, cv2.MORPH_OPEN, kernel, iterations=2)
    
    contours, _ = cv2.findContours(thresh_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        
        if area > 0.10 * width * height:
            planet_mask_out = np.zeros(gray.shape, dtype="uint8")
            cv2.drawContours(planet_mask_out, [largest_contour], -1, 255, thickness=cv2.FILLED)
            
            ((cx, cy), radius) = cv2.minEnclosingCircle(largest_contour)
            print(f"  Planet segmented: Approx. Center=({int(cx)}, {int(cy)}), Radius={int(radius)}")
            return planet_mask_out, int(cx), int(cy), int(radius)
        else:
            print(f"  Largest contour area ({area:.0f}) too small for planet. Segmentation failed.")
    else:
        print("  No contours found for segmentation.")
    return None, None, None, None

def rotate_image_same_canvas(image, angle, center):
    if image is None: return None
    if center is None or center[0] is None or center[1] is None:
        return image 
    (h, w) = image.shape[:2]
    try:
        center_tuple = (float(center[0]), float(center[1]))
        M = cv2.getRotationMatrix2D(center_tuple, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        return rotated
    except Exception as e:
        print(f"  Error during same-canvas rotation: {e}")
        return None

def rotate_image_expanded_canvas_robust(image, angle_degrees, center_xy_original):
    if image is None: return None, center_xy_original
    if center_xy_original is None or \
       center_xy_original[0] is None or center_xy_original[1] is None:
        return image, center_xy_original
    (h_orig, w_orig) = image.shape[:2]
    center_x_orig_float, center_y_orig_float = float(center_xy_original[0]), float(center_xy_original[1])
    M = cv2.getRotationMatrix2D((center_x_orig_float, center_y_orig_float), float(angle_degrees), 1.0)
    corners_orig = np.array([[0, 0], [w_orig - 1, 0], [w_orig - 1, h_orig - 1], [0, h_orig - 1]], dtype="float32")
    ones = np.ones(shape=(len(corners_orig), 1)); points_ones = np.hstack([corners_orig, ones])
    transformed_corners = M @ points_ones.T; transformed_corners = transformed_corners.T
    x_coords, y_coords = transformed_corners[:, 0], transformed_corners[:, 1]
    x_min, x_max = np.min(x_coords), np.max(x_coords); y_min, y_max = np.min(y_coords), np.max(y_coords)
    new_w = math.ceil(x_max - x_min); new_h = math.ceil(y_max - y_min)
    M[0, 2] -= x_min; M[1, 2] -= y_min
    try:
        rotated_image = cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        original_center_homogeneous = np.array([[center_x_orig_float], [center_y_orig_float], [1.0]])
        new_center_transformed = M @ original_center_homogeneous
        new_center_in_expanded = (int(round(new_center_transformed[0,0])), int(round(new_center_transformed[1,0])))
        return rotated_image, new_center_in_expanded
    except Exception as e:
        print(f"  Error during robust expanded canvas rotation: {e}")
        return image, (int(round(center_x_orig_float)), int(round(center_y_orig_float)))

def calculate_band_orientation_metric(image_for_metric, disk_center_in_image, disk_radius):
    if image_for_metric is None or image_for_metric.size == 0: return float('inf')
    if disk_center_in_image is None or disk_radius is None or disk_radius <=0: return float('inf')
    if len(image_for_metric.shape) > 2: gray_image = cv2.cvtColor(image_for_metric, cv2.COLOR_BGR2GRAY)
    else: gray_image = image_for_metric.copy()
    mask = np.zeros(gray_image.shape[:2], dtype="uint8")
    center_pt = (int(round(disk_center_in_image[0])), int(round(disk_center_in_image[1])))
    radius_val = int(round(disk_radius))
    cv2.circle(mask, center_pt, radius_val, 255, -1)
    if np.sum(mask) < 100 : return float('inf')
    masked_gray_image = cv2.bitwise_and(gray_image, gray_image, mask=mask)
    true_points = np.argwhere(mask)
    if true_points.shape[0] == 0: return float('inf')
    top_left, bottom_right = true_points.min(axis=0), true_points.max(axis=0)
    active_region = masked_gray_image[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    if active_region.shape[0] < 2 or active_region.shape[1] < 2: return float('inf')
    diff_x = np.abs(active_region[:, :-1].astype(np.float32) - active_region[:, 1:].astype(np.float32))
    mask_for_diff = mask[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    valid_diff_mask = np.logical_and(mask_for_diff[:, :-1] > 0, mask_for_diff[:, 1:] > 0)
    sum_diff, num_valid_pairs = np.sum(diff_x[valid_diff_mask]), np.sum(valid_diff_mask)
    return (sum_diff / num_valid_pairs) if num_valid_pairs > 0 else float('inf')

def calculate_image_flux(image_data, mask=None):
    if image_data is None: return 0.0
    if len(image_data.shape) == 3 and image_data.shape[2] == 3:
        gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    elif len(image_data.shape) == 2:
        gray = image_data
    else:
        return 0.0
    if mask is not None:
        if mask.shape != gray.shape:
            return np.sum(gray.astype(np.float64))
        return np.sum(gray[mask == 255].astype(np.float64))
    else:
        return np.sum(gray.astype(np.float64))

def get_reference_profile(map_image_path, profile_length=256):
    map_img = cv2.imread(map_image_path, cv2.IMREAD_COLOR)
    if map_img is None: return None
    map_gray = cv2.cvtColor(map_img, cv2.COLOR_BGR2GRAY); profile = np.mean(map_gray, axis=1)
    original_indices = np.linspace(0, 1, len(profile)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, profile); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def get_vertical_profile(image, cx, cy, radius, profile_length=256):
    if image is None or cx is None or cy is None or radius is None or radius <= 0: return None
    if len(image.shape) > 2: gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: gray = image.copy()
    h, w_img = image.shape[:2]; y_start = max(0, int(round(cy - radius))); y_end = min(h, int(round(cy + radius)))
    cx_int = int(round(cx));
    if not (0 <= cx_int < w_img) or y_start >= y_end: return None
    vertical_slice = gray[y_start:y_end, cx_int]
    if vertical_slice.size == 0: return None
    original_indices = np.linspace(0, 1, len(vertical_slice)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, vertical_slice); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def compare_profiles_xcorr(profile1, profile_ref):
    if profile1 is None or profile_ref is None or len(profile1) != len(profile_ref): return 0.0
    p1, pref = np.asarray(profile1), np.asarray(profile_ref); correlation = np.sum(p1 * pref) / len(p1)
    return np.clip(correlation, -1.0, 1.0)

reference_profile = get_reference_profile(REFERENCE_MAP_PATH, profile_length=PROFILE_LENGTH)
if reference_profile is None:
    print("\n!!! WARNING: Could not load reference profile. !!!\n")

print("\n--- Starting Image Processing (v4.6 Shaded Flux Analysis) ---")
input_files = sorted(glob.glob(INPUT_IMAGE_DIR))

if not input_files:
    print(f"Error: No images found: '{INPUT_IMAGE_DIR}'")
else:
    print(f"Found {len(input_files)} images to process...")

total_start_time = time.time()
processed_count = 0

for img_path in input_files:
    img_proc_start_time = time.time()
    filename = os.path.basename(img_path)
    base_output_name = os.path.splitext(filename)[0]
    output_filename_chosen_rotated = base_output_name + '_oriented.png'
    output_path_chosen_rotated = os.path.join(OUTPUT_IMAGE_DIR, output_filename_chosen_rotated)

    print(f"\nProcessing: {filename}")
    img_original = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img_original is None:
        print(f"  Error loading {img_path}. Skipping.")
        continue

    planet_mask, cx_seg, cy_seg, radius_seg = segment_planet_and_get_properties(img_original)
    
    if planet_mask is None or cx_seg is None:
        print(f"  Skipping {filename} - Could not segment planet robustly.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_SEGMENTATION_FAILED.png'), img_original)
        continue
    
    original_pivot_center = (cx_seg, cy_seg) 
    radius_for_analysis = radius_seg

    flux_original_total = calculate_image_flux(img_original)
    print(f"  Flux 1 (Original Total): {flux_original_total:,.0f}")

    flux_masked_on_original = calculate_image_flux(img_original, mask=planet_mask)
    print(f"  Flux 2 (Original Masked Area, R~{radius_for_analysis}): {flux_masked_on_original:,.0f}")

    img_original_with_shading_display = img_original.copy()
    overlay_color_display = np.zeros_like(img_original_with_shading_display)
    overlay_color_display[planet_mask == 255] = [0, 100, 0] 
    alpha_display = 0.4 
    cv2.addWeighted(overlay_color_display, alpha_display, img_original_with_shading_display, 1 - alpha_display, 0, img_original_with_shading_display)
    # Optionally draw contour of the mask for clarity
    contours_for_draw, _ = cv2.findContours(planet_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_for_draw:
        cv2.drawContours(img_original_with_shading_display, contours_for_draw, -1, (0,255,0), 1)


    best_angle_metric = 0
    min_metric = float('inf')
    print(f"  Analyzing rotations for band metric (Center: {original_pivot_center}, R: {radius_for_analysis})...")
    
    for angle_deg_test in range(0, 360, ROTATION_STEP):
        temp_rotated_same_canvas = rotate_image_same_canvas(img_original, angle_deg_test, original_pivot_center)
        if temp_rotated_same_canvas is None: continue
        metric = calculate_band_orientation_metric(temp_rotated_same_canvas, original_pivot_center, radius_for_analysis)
        if metric < min_metric:
            min_metric = metric
            best_angle_metric = angle_deg_test
            
    if min_metric == float('inf'):
        print(f"  Failed to find optimal rotation for {filename} via metric. Saving original.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_METRIC_FAILED.png'), img_original)
        continue
    print(f"  Initial metric-based best angle: {best_angle_metric}° (Metric: {min_metric:.3f})")

    angle_opt1 = best_angle_metric
    angle_opt2 = (best_angle_metric + 180) % 360

    img_expanded_rotated_opt1, center_in_opt1_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt1, original_pivot_center)
    img_expanded_rotated_opt2, center_in_opt2_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt2, original_pivot_center)

    if img_expanded_rotated_opt1 is None or img_expanded_rotated_opt2 is None:
        print(f"  Error during final full-view rotations for {filename}. Saving original.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_FINAL_ROT_FAILED.png'), img_original)
        continue

    print("  --- Profile Comparison Diagnostic ---")
    profile_suggestion = "Profile Check Failed or Ref Missing"
    chosen_angle_final = angle_opt1 

    if reference_profile is not None:
        profile1 = get_vertical_profile(img_expanded_rotated_opt1, center_in_opt1_canvas[0], center_in_opt1_canvas[1], radius_for_analysis, PROFILE_LENGTH)
        profile2 = get_vertical_profile(img_expanded_rotated_opt2, center_in_opt2_canvas[0], center_in_opt2_canvas[1], radius_for_analysis, PROFILE_LENGTH)

        if profile1 is not None and profile2 is not None:
            score1 = compare_profiles_xcorr(profile1, reference_profile)
            score2 = compare_profiles_xcorr(profile2, reference_profile)
            print(f"    Profile Score: Angle {angle_opt1}° (Opt1) = {score1:.4f}")
            print(f"    Profile Score: Angle {angle_opt2}° (Opt2) = {score2:.4f}")
            if score1 >= score2: chosen_angle_final = angle_opt1; profile_suggestion = f"Suggests Angle {angle_opt1}° (Score: {score1:.4f})"
            else: chosen_angle_final = angle_opt2; profile_suggestion = f"Suggests Angle {angle_opt2}° (Score: {score2:.4f})"
        else:
            print("    Could not extract profiles for comparison. Using metric angle.")
            profile_suggestion = f"Profile Extraction Failed. Using metric Angle {chosen_angle_final}°"
    else:
        print("    Reference profile not available. Using metric-based angle.")
        profile_suggestion = f"No Ref Profile. Using metric Angle {chosen_angle_final}°"
        
    print(f"    {profile_suggestion}")
    print(f"  Final Chosen Angle for {filename}: {chosen_angle_final} degrees")

    img_chosen_full_rotated_expanded_display, center_in_chosen_expanded_canvas = \
        rotate_image_expanded_canvas_robust(img_original, chosen_angle_final, original_pivot_center)
    
    if img_chosen_full_rotated_expanded_display is None:
        print(f"  ERROR: Final chosen rotation for display failed for {filename}.")
        img_chosen_full_rotated_expanded_display = img_original 
        flux_rotated_total = flux_original_total 
        print("         Displaying original image instead for Panel 3.")
    else:
        flux_rotated_total = calculate_image_flux(img_chosen_full_rotated_expanded_display)

    print(f"  Flux 3 (Rotated Total, Expanded Canvas): {flux_rotated_total:,.0f}")
    flux_diff = flux_original_total - flux_rotated_total
    flux_diff_percent = (flux_diff / flux_original_total * 100) if flux_original_total != 0 else 0
    print(f"  Flux Difference (Original - Rotated_Expanded): {flux_diff:,.0f} ({flux_diff_percent:.2f}%)")

    print(f"  Saving Chosen Full-View Orientation (Angle {chosen_angle_final} deg) to: {output_path_chosen_rotated}")
    save_success = cv2.imwrite(output_path_chosen_rotated, img_chosen_full_rotated_expanded_display)
    if not save_success: print(f"  Warning: Failed to save {output_path_chosen_rotated}.")

    if SHOW_PLOTS:
        fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 

        axes[0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"1. Original: {filename}\nFlux: {flux_original_total:,.0f}", fontsize=10)
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(img_original_with_shading_display, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"2. Original + Planet Shaded\nFlux (Masked): {flux_masked_on_original:,.0f}", fontsize=10)
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(img_chosen_full_rotated_expanded_display, cv2.COLOR_BGR2RGB))
        axes[2].set_title(f"3. Full Rotated (Angle {chosen_angle_final}°)\nFlux: {flux_rotated_total:,.0f}\n{profile_suggestion}", fontsize=10)
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()

    processed_count += 1
    img_proc_elapsed_time = time.time() - img_proc_start_time
    print(f"  Time taken for {filename}: {img_proc_elapsed_time:.2f} seconds.")

total_elapsed_time = time.time() - total_start_time
print("\n--- Processing finished ---")
if processed_count > 0:
    print(f"Successfully processed {processed_count} out of {len(input_files)} images.")
    print(f"Total time: {total_elapsed_time:.2f} seconds.")
    print(f"Average time per image: {total_elapsed_time / processed_count:.2f} seconds.")
else:
    print(f"No images were successfully processed from {len(input_files)} found.")
print(f"Output saved in: {os.path.abspath(OUTPUT_IMAGE_DIR)}")


In [ ]:
import cv2
import numpy as np
import os
import glob
import math
import matplotlib.pyplot as plt
import time

INPUT_IMAGE_DIR = 'images/4 Cropped_Images/crop_157_20190721_140130.png'
OUTPUT_IMAGE_DIR = 'images/testing'
REFERENCE_MAP_PATH = 'T2.jpg'

ROTATION_STEP = 1 
PROFILE_LENGTH = 256 
SHOW_PLOTS = True
HIGH_CORRELATION_THRESHOLD = 0.88 # Threshold for pre-check

os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

print("--- Configuration Summary (v4.7 Pre-check Rotation) ---")
print(f"Input path/pattern: {INPUT_IMAGE_DIR}")
print(f"Output directory:   {os.path.abspath(OUTPUT_IMAGE_DIR)}")
print(f"Reference Map:      {REFERENCE_MAP_PATH}")
print(f"Rotation Step:      {ROTATION_STEP} degrees")
print(f"Pre-check Threshold:{HIGH_CORRELATION_THRESHOLD}")
print(f"Show Plots:         {SHOW_PLOTS}")
print("-" * 40)

if not os.path.exists(REFERENCE_MAP_PATH):
    print(f"\n!!! CRITICAL WARNING: Reference map '{REFERENCE_MAP_PATH}' not found. !!!\n")

def segment_planet_and_get_properties(image_color):
    if image_color is None: return None, None, None, None
    img_for_segmentation = image_color.copy()
    if len(img_for_segmentation.shape) > 2 and img_for_segmentation.shape[2] == 3:
        gray = cv2.cvtColor(img_for_segmentation, cv2.COLOR_BGR2GRAY)
    elif len(img_for_segmentation.shape) == 2:
        gray = img_for_segmentation
    else:
        return None, None, None, None
    height, width = gray.shape
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    block_size = int(min(height, width) / 15); 
    if block_size % 2 == 0: block_size += 1
    if block_size <= 1: block_size = 11 
    C_value = 2 
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, C_value)
    if np.mean(thresh[blurred > np.mean(blurred)]) < 127 and np.mean(blurred) > 30:
         thresh = cv2.bitwise_not(thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    thresh_cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh_cleaned = cv2.morphologyEx(thresh_cleaned, cv2.MORPH_OPEN, kernel, iterations=2)
    contours, _ = cv2.findContours(thresh_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        if area > 0.10 * width * height:
            planet_mask_out = np.zeros(gray.shape, dtype="uint8")
            cv2.drawContours(planet_mask_out, [largest_contour], -1, 255, thickness=cv2.FILLED)
            ((cx, cy), radius) = cv2.minEnclosingCircle(largest_contour)
            return planet_mask_out, int(cx), int(cy), int(radius)
    return None, None, None, None

def rotate_image_same_canvas(image, angle, center):
    if image is None: return None
    if center is None or center[0] is None or center[1] is None: return image 
    (h, w) = image.shape[:2]
    try:
        center_tuple = (float(center[0]), float(center[1]))
        M = cv2.getRotationMatrix2D(center_tuple, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        return rotated
    except Exception: return None

def rotate_image_expanded_canvas_robust(image, angle_degrees, center_xy_original):
    if image is None: return None, center_xy_original
    if center_xy_original is None or center_xy_original[0] is None or center_xy_original[1] is None: return image, center_xy_original
    (h_orig, w_orig) = image.shape[:2]
    center_x_orig_float, center_y_orig_float = float(center_xy_original[0]), float(center_xy_original[1])
    M = cv2.getRotationMatrix2D((center_x_orig_float, center_y_orig_float), float(angle_degrees), 1.0)
    corners_orig = np.array([[0, 0], [w_orig - 1, 0], [w_orig - 1, h_orig - 1], [0, h_orig - 1]], dtype="float32")
    ones = np.ones(shape=(len(corners_orig), 1)); points_ones = np.hstack([corners_orig, ones])
    transformed_corners = M @ points_ones.T; transformed_corners = transformed_corners.T
    x_coords, y_coords = transformed_corners[:, 0], transformed_corners[:, 1]
    x_min, x_max = np.min(x_coords), np.max(x_coords); y_min, y_max = np.min(y_coords), np.max(y_coords)
    new_w = math.ceil(x_max - x_min); new_h = math.ceil(y_max - y_min)
    M[0, 2] -= x_min; M[1, 2] -= y_min
    try:
        rotated_image = cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
        original_center_homogeneous = np.array([[center_x_orig_float], [center_y_orig_float], [1.0]])
        new_center_transformed = M @ original_center_homogeneous
        new_center_in_expanded = (int(round(new_center_transformed[0,0])), int(round(new_center_transformed[1,0])))
        return rotated_image, new_center_in_expanded
    except Exception: return image, (int(round(center_x_orig_float)), int(round(center_y_orig_float)))

def calculate_band_orientation_metric(image_for_metric, disk_center_in_image, disk_radius):
    if image_for_metric is None or image_for_metric.size == 0: return float('inf')
    if disk_center_in_image is None or disk_radius is None or disk_radius <=0: return float('inf')
    if len(image_for_metric.shape) > 2: gray_image = cv2.cvtColor(image_for_metric, cv2.COLOR_BGR2GRAY)
    else: gray_image = image_for_metric.copy()
    mask = np.zeros(gray_image.shape[:2], dtype="uint8")
    center_pt = (int(round(disk_center_in_image[0])), int(round(disk_center_in_image[1])))
    radius_val = int(round(disk_radius))
    cv2.circle(mask, center_pt, radius_val, 255, -1)
    if np.sum(mask) < 100 : return float('inf')
    masked_gray_image = cv2.bitwise_and(gray_image, gray_image, mask=mask)
    true_points = np.argwhere(mask)
    if true_points.shape[0] == 0: return float('inf')
    top_left, bottom_right = true_points.min(axis=0), true_points.max(axis=0)
    active_region = masked_gray_image[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    if active_region.shape[0] < 2 or active_region.shape[1] < 2: return float('inf')
    diff_x = np.abs(active_region[:, :-1].astype(np.float32) - active_region[:, 1:].astype(np.float32))
    mask_for_diff = mask[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    valid_diff_mask = np.logical_and(mask_for_diff[:, :-1] > 0, mask_for_diff[:, 1:] > 0)
    sum_diff, num_valid_pairs = np.sum(diff_x[valid_diff_mask]), np.sum(valid_diff_mask)
    return (sum_diff / num_valid_pairs) if num_valid_pairs > 0 else float('inf')

def calculate_image_flux(image_data, mask=None):
    if image_data is None: return 0.0
    if len(image_data.shape) == 3 and image_data.shape[2] == 3:
        gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    elif len(image_data.shape) == 2:
        gray = image_data
    else: return 0.0
    if mask is not None:
        if mask.shape != gray.shape: return np.sum(gray.astype(np.float64))
        return np.sum(gray[mask == 255].astype(np.float64))
    else: return np.sum(gray.astype(np.float64))

def get_reference_profile(map_image_path, profile_length=256):
    map_img = cv2.imread(map_image_path, cv2.IMREAD_COLOR)
    if map_img is None: return None
    map_gray = cv2.cvtColor(map_img, cv2.COLOR_BGR2GRAY); profile = np.mean(map_gray, axis=1)
    original_indices = np.linspace(0, 1, len(profile)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, profile); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def get_vertical_profile(image, cx, cy, radius, profile_length=256):
    if image is None or cx is None or cy is None or radius is None or radius <= 0: return None
    if len(image.shape) > 2: gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else: gray = image.copy()
    h, w_img = image.shape[:2]; y_start = max(0, int(round(cy - radius))); y_end = min(h, int(round(cy + radius)))
    cx_int = int(round(cx));
    if not (0 <= cx_int < w_img) or y_start >= y_end: return None
    vertical_slice = gray[y_start:y_end, cx_int]
    if vertical_slice.size == 0: return None
    original_indices = np.linspace(0, 1, len(vertical_slice)); target_indices = np.linspace(0, 1, profile_length)
    resized_profile = np.interp(target_indices, original_indices, vertical_slice); profile_std = np.std(resized_profile)
    if profile_std > 1e-6: normalized_profile = (resized_profile - np.mean(resized_profile)) / profile_std
    else: normalized_profile = np.zeros_like(resized_profile)
    return normalized_profile

def compare_profiles_xcorr(profile1, profile_ref):
    if profile1 is None or profile_ref is None or len(profile1) != len(profile_ref): return 0.0
    p1, pref = np.asarray(profile1), np.asarray(profile_ref); correlation = np.sum(p1 * pref) / len(p1)
    return np.clip(correlation, -1.0, 1.0)

reference_profile = get_reference_profile(REFERENCE_MAP_PATH, profile_length=PROFILE_LENGTH)
if reference_profile is None:
    print("\n!!! WARNING: Reference profile not loaded. Orientation checks will be limited. !!!\n")

print("\n--- Starting Image Processing (v4.7 Pre-check Rotation) ---")
input_files = sorted(glob.glob(INPUT_IMAGE_DIR))

if not input_files:
    print(f"Error: No images found: '{INPUT_IMAGE_DIR}'")
else:
    print(f"Found {len(input_files)} images to process...")

total_start_time = time.time()
processed_count = 0

for img_path in input_files:
    img_proc_start_time = time.time()
    filename = os.path.basename(img_path)
    base_output_name = os.path.splitext(filename)[0]
    output_filename_chosen_rotated = base_output_name + '_chosen_shaded_rotated_fullview.png'
    output_path_chosen_rotated = os.path.join(OUTPUT_IMAGE_DIR, output_filename_chosen_rotated)

    print(f"\nProcessing: {filename}")
    img_original = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img_original is None:
        print(f"  Error loading {img_path}. Skipping.")
        continue

    planet_mask, cx_orig, cy_orig, radius_orig = segment_planet_and_get_properties(img_original)
    if planet_mask is None or cx_orig is None:
        print(f"  Skipping {filename} - Could not segment planet.")
        cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_SEGMENTATION_FAILED.png'), img_original)
        continue
    
    original_pivot_center = (cx_orig, cy_orig) 

    flux_original_total = calculate_image_flux(img_original)
    print(f"  Flux 1 (Original Total): {flux_original_total:,.0f}")

    flux_masked_on_original = calculate_image_flux(img_original, mask=planet_mask)
    print(f"  Flux 2 (Original Masked Area, R~{radius_orig}): {flux_masked_on_original:,.0f}")

    img_original_with_shading_display = img_original.copy()
    overlay_color_display = np.zeros_like(img_original_with_shading_display)
    overlay_color_display[planet_mask == 255] = [0, 100, 0] 
    alpha_display = 0.4 
    cv2.addWeighted(overlay_color_display, alpha_display, img_original_with_shading_display, 1 - alpha_display, 0, img_original_with_shading_display)
    contours_for_draw, _ = cv2.findContours(planet_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_for_draw:
        cv2.drawContours(img_original_with_shading_display, contours_for_draw, -1, (0,255,0), 1)

    chosen_angle_final = 0
    profile_suggestion = "No Profile Check Done"
    rotation_was_needed = True 
    img_chosen_full_rotated_expanded_display = None
    center_in_chosen_expanded_canvas = None

    if reference_profile is not None:
        print("  Performing pre-check using profile comparison for 0 and 180 deg...")
        profile_0 = get_vertical_profile(img_original, cx_orig, cy_orig, radius_orig, PROFILE_LENGTH)
        score_0 = compare_profiles_xcorr(profile_0, reference_profile) if profile_0 is not None else -1.0

        img_180_temp, center_180_temp = rotate_image_expanded_canvas_robust(img_original, 180, original_pivot_center)
        if img_180_temp is not None:
            profile_180 = get_vertical_profile(img_180_temp, center_180_temp[0], center_180_temp[1], radius_orig, PROFILE_LENGTH)
            score_180 = compare_profiles_xcorr(profile_180, reference_profile) if profile_180 is not None else -1.0
        else:
            score_180 = -1.0
            print("    Warning: Failed to rotate for 180 deg pre-check.")

        print(f"    Pre-check Score (0 deg): {score_0:.4f}, Score (180 deg): {score_180:.4f}")

        if score_0 >= HIGH_CORRELATION_THRESHOLD and score_0 >= score_180 :
            chosen_angle_final = 0
            profile_suggestion = f"Pre-check: No Rotation (Angle 0°, Score: {score_0:.4f})"
            rotation_was_needed = False
            img_chosen_full_rotated_expanded_display = img_original.copy() # No rotation, use original
            center_in_chosen_expanded_canvas = original_pivot_center
            print(f"  Rotation not required for this image (0° is good). {profile_suggestion}")
        elif score_180 >= HIGH_CORRELATION_THRESHOLD and score_180 > score_0:
            chosen_angle_final = 180
            profile_suggestion = f"Pre-check: Needs 180° Flip (Score: {score_180:.4f})"
            rotation_was_needed = False # Technically a rotation, but not from full search
            img_chosen_full_rotated_expanded_display = img_180_temp
            center_in_chosen_expanded_canvas = center_180_temp
            print(f"  Minimal rotation (180° flip) chosen based on pre-check. {profile_suggestion}")
    
    if rotation_was_needed:
        print(f"  Pre-check did not meet threshold or ref_profile missing. Proceeding with full metric scan...")
        best_angle_metric = 0
        min_metric = float('inf')
        
        for angle_deg_test in range(0, 360, ROTATION_STEP):
            temp_rotated_same_canvas = rotate_image_same_canvas(img_original, angle_deg_test, original_pivot_center)
            if temp_rotated_same_canvas is None: continue
            metric = calculate_band_orientation_metric(temp_rotated_same_canvas, original_pivot_center, radius_orig)
            if metric < min_metric:
                min_metric = metric
                best_angle_metric = angle_deg_test
                
        if min_metric == float('inf'):
            print(f"  Failed to find optimal rotation for {filename} via metric. Saving original.")
            cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_METRIC_FAILED.png'), img_original)
            continue
        print(f"  Metric-based best angle: {best_angle_metric}° (Metric: {min_metric:.3f})")

        angle_opt1 = best_angle_metric
        angle_opt2 = (best_angle_metric + 180) % 360

        img_expanded_rotated_opt1, center_in_opt1_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt1, original_pivot_center)
        img_expanded_rotated_opt2, center_in_opt2_canvas = rotate_image_expanded_canvas_robust(img_original, angle_opt2, original_pivot_center)

        if img_expanded_rotated_opt1 is None or img_expanded_rotated_opt2 is None:
            print(f"  Error during full-view rotations for profile candidates. Saving original.")
            cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, base_output_name + '_FINAL_ROT_FAILED.png'), img_original)
            continue

        print("  --- Profile Comparison (Post-Metric Scan) ---")
        chosen_angle_final = angle_opt1 

        if reference_profile is not None:
            profile1 = get_vertical_profile(img_expanded_rotated_opt1, center_in_opt1_canvas[0], center_in_opt1_canvas[1], radius_orig, PROFILE_LENGTH)
            profile2 = get_vertical_profile(img_expanded_rotated_opt2, center_in_opt2_canvas[0], center_in_opt2_canvas[1], radius_orig, PROFILE_LENGTH)

            if profile1 is not None and profile2 is not None:
                score1 = compare_profiles_xcorr(profile1, reference_profile)
                score2 = compare_profiles_xcorr(profile2, reference_profile)
                print(f"    Profile Score: Angle {angle_opt1}° (Opt1) = {score1:.4f}")
                print(f"    Profile Score: Angle {angle_opt2}° (Opt2) = {score2:.4f}")
                if score1 >= score2: chosen_angle_final = angle_opt1; profile_suggestion = f"Suggests Angle {angle_opt1}° (Score: {score1:.4f})"
                else: chosen_angle_final = angle_opt2; profile_suggestion = f"Suggests Angle {angle_opt2}° (Score: {score2:.4f})"
            else:
                profile_suggestion = f"Profile Extraction Failed. Metric Angle {chosen_angle_final}°"
        else:
            profile_suggestion = f"No Ref Profile. Metric Angle {chosen_angle_final}°"
        
        print(f"    {profile_suggestion}")
        print(f"  Final Chosen Angle for {filename} (post-metric): {chosen_angle_final} degrees")
        
        img_chosen_full_rotated_expanded_display, center_in_chosen_expanded_canvas = \
            rotate_image_expanded_canvas_robust(img_original, chosen_angle_final, original_pivot_center)

    if img_chosen_full_rotated_expanded_display is None:
        img_chosen_full_rotated_expanded_display = img_original 
        flux_rotated_total = flux_original_total 
        print(f"  ERROR: Final chosen rotation for display resulted in None. Displaying original for Panel 3.")
    else:
        flux_rotated_total = calculate_image_flux(img_chosen_full_rotated_expanded_display)

    print(f"  Flux 3 (Rotated Total, Expanded Canvas): {flux_rotated_total:,.0f}")
    flux_diff = flux_original_total - flux_rotated_total
    flux_diff_percent = (flux_diff / flux_original_total * 100) if flux_original_total != 0 else 0
    print(f"  Flux Difference (Original - Rotated_Expanded): {flux_diff:,.0f} ({flux_diff_percent:.2f}%)")

    print(f"  Saving Chosen Full-View Orientation (Angle {chosen_angle_final} deg) to: {output_path_chosen_rotated}")
    save_success = cv2.imwrite(output_path_chosen_rotated, img_chosen_full_rotated_expanded_display)
    if not save_success: print(f"  Warning: Failed to save {output_path_chosen_rotated}.")

    if SHOW_PLOTS:
        fig, axes = plt.subplots(1, 3, figsize=(24, 8)) 

        axes[0].imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"1. Original: {filename}\nFlux: {flux_original_total:,.0f}", fontsize=10)
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(img_original_with_shading_display, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"2. Original + Planet Shaded\nFlux (Masked): {flux_masked_on_original:,.0f}", fontsize=10)
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(img_chosen_full_rotated_expanded_display, cv2.COLOR_BGR2RGB))
        if not rotation_was_needed and chosen_angle_final == 0:
             axes[2].set_title(f"3. Full Rotated (Angle {chosen_angle_final}° - No Rotation Needed)\nFlux: {flux_rotated_total:,.0f}\n{profile_suggestion}", fontsize=10)
        elif not rotation_was_needed and chosen_angle_final == 180:
            axes[2].set_title(f"3. Full Rotated (Angle {chosen_angle_final}° - Flipped)\nFlux: {flux_rotated_total:,.0f}\n{profile_suggestion}", fontsize=10)
        else:
            axes[2].set_title(f"3. Full Rotated (Angle {chosen_angle_final}°)\nFlux: {flux_rotated_total:,.0f}\n{profile_suggestion}", fontsize=10)
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()

    processed_count += 1
    img_proc_elapsed_time = time.time() - img_proc_start_time
    print(f"  Time taken for {filename}: {img_proc_elapsed_time:.2f} seconds.")

total_elapsed_time = time.time() - total_start_time
print("\n--- Processing finished ---")
if processed_count > 0:
    print(f"Successfully processed {processed_count} out of {len(input_files)} images.")
    print(f"Total time: {total_elapsed_time:.2f} seconds.")
    print(f"Average time per image: {total_elapsed_time / processed_count:.2f} seconds.")
else:
    print(f"No images were successfully processed from {len(input_files)} found.")
print(f"Output saved in: {os.path.abspath(OUTPUT_IMAGE_DIR)}")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, exposure, feature, measure, morphology, restoration
from scipy.signal import find_peaks
from scipy.fft import rfft, rfftfreq

# --- Configuration ---
IMAGE_PATH = 'crop_001_20150119_021648.png' # Path to your image
# Preprocessing
GAUSSIAN_BLUR_KERNEL = (3, 3) # Kernel size for slight noise reduction
CLAHE_CLIP_LIMIT = 1.5        # Contrast limit for CLAHE
CLAHE_TILE_GRID_SIZE = (8, 8) # Tile grid size for CLAHE
# Disk Detection
DISK_THRESHOLD_VALUE = 20    # Initial guess for thresholding to find the disk contour (adjust if needed)
MIN_DISK_AREA_FACTOR = 0.5   # Minimum area for the disk contour relative to image area
# Band Analysis
PROFILE_AVERAGING_WIDTH_FACTOR = 0.3 # Fraction of disk diameter for central profile averaging
EDGE_DETECTION_SIGMA = 1.0          # Sigma for Canny edge detector
# Moon Shadow Detection
SHADOW_MAX_THRESHOLD = 40     # Pixels darker than this might be part of the shadow
SHADOW_MIN_AREA = 50          # Minimum pixel area for a detected shadow blob
SHADOW_MAX_AREA = 2000        # Maximum pixel area (helps filter noise)

# --- 1. Load and Preprocess Image ---
print("Step 1: Loading and Preprocessing...")
try:
    # Load image using skimage (handles different formats well), force grayscale
    image_orig = io.imread(IMAGE_PATH, as_gray=True)
    # Convert to uint8 if it's not already (OpenCV often prefers this)
    if image_orig.dtype != np.uint8:
        image_uint8 = (image_orig * 255).astype(np.uint8)
    else:
        image_uint8 = image_orig

    print(f"Image loaded successfully. Shape: {image_uint8.shape}, dtype: {image_uint8.dtype}")

except FileNotFoundError:
    print(f"ERROR: Image file not found at {IMAGE_PATH}")
    exit()
except Exception as e:
    print(f"ERROR: Could not load image. {e}")
    exit()

# Apply slight Gaussian Blur for noise reduction
image_blurred = cv2.GaussianBlur(image_uint8, GAUSSIAN_BLUR_KERNEL, 0)

# Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP_LIMIT, tileGridSize=CLAHE_TILE_GRID_SIZE)
image_clahe = clahe.apply(image_blurred)

# --- 2. Detect Jupiter's Disk ---
print("Step 2: Detecting Jupiter's Disk...")
# Thresholding to isolate the planet from the black background
_, thresh = cv2.threshold(image_blurred, DISK_THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)

# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour (should be the planet disk)
largest_contour = None
max_area = 0
img_height, img_width = image_uint8.shape
min_required_area = MIN_DISK_AREA_FACTOR * img_height * img_width / (np.pi / 4) # Heuristic min area

for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > max_area:
        max_area = area
        largest_contour = cnt

if largest_contour is None or max_area < min_required_area:
    print("ERROR: Could not reliably detect Jupiter's disk contour. Adjust DISK_THRESHOLD_VALUE or check image.")
    # Fallback: Assume the image is mostly filled by the disk (less accurate)
    center_x, center_y = img_width // 2, img_height // 2
    radius = min(center_x, center_y)
    disk_mask = np.zeros_like(image_uint8)
    cv2.circle(disk_mask, (center_x, center_y), radius, 255, -1)
    print("Warning: Using fallback disk estimation.")
else:
    # Get the minimum enclosing circle for the largest contour
    (center_x_f, center_y_f), radius_f = cv2.minEnclosingCircle(largest_contour)
    center_x, center_y = int(center_x_f), int(center_y_f)
    radius = int(radius_f)
    print(f"Disk detected: Center=({center_x}, {center_y}), Radius={radius}")

    # Create a precise mask from the enclosing circle
    disk_mask = np.zeros_like(image_uint8)
    cv2.circle(disk_mask, (center_x, center_y), radius, 255, -1)

# Apply the mask to the enhanced image
image_masked = cv2.bitwise_and(image_clahe, image_clahe, mask=disk_mask)

# --- 3. Band Analysis ---
print("Step 3: Performing Band Analysis...")

# 3a. Intensity Profile (Corrected for Disk Shape)
print("  - Calculating Intensity Profile...")
latitudinal_profile = []
y_coords = []
img_height, img_width = image_clahe.shape # Get image dimensions

# --- Corrected loop range ---
# Calculate bounds, clamping them to the image dimensions [0, img_height)
y_start_loop = max(0, center_y - radius)
y_end_loop = min(img_height, center_y + radius) # Stop before img_height

print(f"  - Looping through y-coordinates from {y_start_loop} to {y_end_loop-1}") # Debug print

# Iterate through horizontal lines (approximating latitudes) within the disk boundaries
for y in range(y_start_loop, y_end_loop): # Use the clamped range
    # Calculate the start and end x-coordinates for this row within the circle
    # Note: dx calculation might still yield negative for y at the very edge
    # if radius calculation was slightly off, but `if dx >= 0` handles that.
    dx = radius**2 - (y - center_y)**2
    if dx >= 0: # Ensure the row intersects the circle mathematically
        half_width = int(np.sqrt(dx))
        x_start = max(0, center_x - half_width)
        x_end = min(img_width, center_x + half_width)

        if x_start < x_end: # Ensure there are pixels on this line within image width
            # Extract pixel values within the disk on this row
            row_pixels = image_clahe[y, x_start:x_end] # This line should now be safe
            # Calculate the average intensity for this latitude
            avg_intensity = np.mean(row_pixels)
            latitudinal_profile.append(avg_intensity)
            y_coords.append(y) # Store corresponding y coordinate
# --- End of corrected section ---

latitudinal_profile = np.array(latitudinal_profile)
y_coords = np.array(y_coords)

# Find peaks (Zones) and troughs (Belts) in the profile
# Adjust prominence based on expected contrast between bands
peaks, _ = find_peaks(latitudinal_profile, prominence=5)
troughs, _ = find_peaks(-latitudinal_profile, prominence=5)


# 3b. Frequency Analysis (FFT) of the profile
print("  - Performing FFT on Intensity Profile...")
N = len(latitudinal_profile)
if N > 1:
    yf = rfft(latitudinal_profile - np.mean(latitudinal_profile)) # Remove DC component
    xf = rfftfreq(N, 1) # Frequency bins (cycles per pixel distance)

    # Find dominant frequencies (excluding near-zero frequency)
    fft_peaks, _ = find_peaks(np.abs(yf)[1:], height=np.max(np.abs(yf)[1:])*0.1) # Find peaks above 10% of max
    fft_peaks += 1 # Adjust index back
    dominant_freqs = xf[fft_peaks]
    dominant_periods = 1.0 / dominant_freqs[dominant_freqs > 0] # Convert frequency to pixel period
    print(f"  - Dominant band spatial periods (pixels): {dominant_periods}")
else:
    print("  - Not enough data points for FFT analysis.")
    xf, yf, dominant_periods = None, None, []


# 3c. Edge Detection (Canny)
print("  - Performing Edge Detection...")
edges = cv2.Canny(image_masked, threshold1=50, threshold2=150) # Adjust thresholds as needed
# Alternative using skimage (sometimes gives different results)
# edges_sk = feature.canny(image_masked, sigma=EDGE_DETECTION_SIGMA)

# --- 4. Moon Shadow Detection ---
print("Step 4: Detecting Moon Shadow...")
# Use the masked, contrast-enhanced image for detection
# Simple Thresholding for very dark spots
_, shadow_thresh = cv2.threshold(image_masked, SHADOW_MAX_THRESHOLD, 255, cv2.THRESH_BINARY_INV) # Inverse threshold

# Optional: Morphological opening to remove small noise points
kernel = morphology.disk(2)
# Ensure this line assigns to 'shadow_thresh_opened'
shadow_thresh_opened = morphology.binary_opening(shadow_thresh, kernel) # Returns boolean

# --- Corrected conversion line ---
# Convert boolean mask back to uint8 (0 and 255) for OpenCV
# Use the variable that holds the result from the line above:
shadow_thresh_opened_uint8 = (shadow_thresh_opened.astype(np.uint8)) * 255
print(f"  - Converted shadow mask dtype: {shadow_thresh_opened_uint8.dtype}") # Debug print

# Find contours in the thresholded image (use the converted uint8 image)
shadow_contours, _ = cv2.findContours(shadow_thresh_opened_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

detected_shadows = []
shadow_details = []
for cnt in shadow_contours:
    area = cv2.contourArea(cnt)
    if SHADOW_MIN_AREA < area < SHADOW_MAX_AREA:
        # Calculate moments to find centroid
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            scx = int(M["m10"] / M["m00"])
            scy = int(M["m01"] / M["m00"])
            # Get bounding box and approximate radius
            x, y, w, h = cv2.boundingRect(cnt)
            approx_radius = (w + h) / 4.0
            # Calculate circularity ( Solidity = Contour Area / Convex Hull Area ) might be better
            hull = cv2.convexHull(cnt)
            hull_area = cv2.contourArea(hull)
            solidity = area / hull_area if hull_area > 0 else 0

            details = {
                'contour': cnt,
                'center': (scx, scy),
                'area': area,
                'radius_approx': approx_radius,
                'solidity': solidity
            }
            shadow_details.append(details)
            detected_shadows.append(cnt)
            print(f"  - Detected shadow candidate: Center=({scx}, {scy}), Area={area:.1f}, Solidity={solidity:.2f}")

if not detected_shadows:
    print("  - No significant moon shadow detected matching criteria.")


# --- 5. Visualization ---
print("Step 5: Generating Visualizations...")
plt.style.use('default') # Use default style for clarity

# Figure 1: Input and Preprocessing steps
fig1, axes1 = plt.subplots(1, 4, figsize=(20, 5))
fig1.suptitle('Image Loading and Preprocessing', fontsize=16)
axes1[0].imshow(image_uint8, cmap='gray')
axes1[0].set_title('Original Grayscale')
axes1[0].axis('off')
axes1[1].imshow(image_blurred, cmap='gray')
axes1[1].set_title('Blurred')
axes1[1].axis('off')
axes1[2].imshow(image_clahe, cmap='gray')
axes1[2].set_title('CLAHE Enhanced')
axes1[2].axis('off')
axes1[3].imshow(disk_mask, cmap='gray')
axes1[3].set_title('Detected Disk Mask')
axes1[3].axis('off')
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap

# Figure 2: Band Analysis Results
fig2, axes2 = plt.subplots(1, 3, figsize=(20, 6))
fig2.suptitle('Band Analysis', fontsize=16)

# Plot Enhanced Image with Disk Outline
axes2[0].imshow(image_clahe, cmap='gray')
axes2[0].add_patch(plt.Circle((center_x, center_y), radius, color='cyan', fill=False, lw=1))
axes2[0].set_title('Enhanced Image + Disk Outline')
axes2[0].axis('off')

# Plot Intensity Profile with detected zones/belts
axes2[1].plot(latitudinal_profile, y_coords, label='Intensity Profile', color='blue')
axes2[1].plot(latitudinal_profile[peaks], y_coords[peaks], "x", color='red', label='Zones (Peaks)')
axes2[1].plot(latitudinal_profile[troughs], y_coords[troughs], "o", markerfacecolor='none', markeredgecolor='green', label='Belts (Troughs)')
axes2[1].set_xlabel('Average Pixel Intensity')
axes2[1].set_ylabel('Y Coordinate (Pixel)')
axes2[1].set_title('Latitudinal Intensity Profile')
axes2[1].invert_yaxis() # Match image orientation
axes2[1].grid(True, linestyle=':')
axes2[1].legend(fontsize='small')

# Plot FFT Magnitude Spectrum
if xf is not None and yf is not None:
    axes2[2].plot(xf[1:], np.abs(yf[1:]), label='FFT Magnitude', color='purple') # Skip DC component (index 0)
    if len(dominant_periods) > 0:
       # Mark dominant frequencies/periods
       axes2[2].plot(xf[fft_peaks], np.abs(yf[fft_peaks]), "x", color='orange', label=f'Peaks (Periods ~{np.min(dominant_periods):.1f}-{np.max(dominant_periods):.1f} px)')
    axes2[2].set_xlabel('Spatial Frequency (cycles/pixel)')
    axes2[2].set_ylabel('Magnitude')
    axes2[2].set_title('FFT of Intensity Profile')
    axes2[2].grid(True, linestyle=':')
    axes2[2].legend(fontsize='small')
    axes2[2].set_yscale('log') # Often helps visualize smaller peaks
else:
    axes2[2].text(0.5, 0.5, 'FFT Analysis Skipped', ha='center', va='center')
    axes2[2].set_title('FFT of Intensity Profile')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Figure 3: Edge Detection and Shadow Detection
fig3, axes3 = plt.subplots(1, 3, figsize=(18, 6))
fig3.suptitle('Feature Detection', fontsize=16)

axes3[0].imshow(edges, cmap='gray')
axes3[0].set_title('Band Edges (Canny)')
axes3[0].axis('off')

axes3[1].imshow(shadow_thresh_opened, cmap='gray')
axes3[1].set_title('Moon Shadow Threshold Mask')
axes3[1].axis('off')

# Draw results on the CLAHE image
image_results = cv2.cvtColor(image_clahe, cv2.COLOR_GRAY2BGR) # Convert to color for drawing
# Draw disk outline
cv2.circle(image_results, (center_x, center_y), radius, (0, 255, 255), 1) # Cyan circle
# Draw detected shadow contours
cv2.drawContours(image_results, detected_shadows, -1, (0, 255, 0), 2) # Green contours
# Label detected shadow centers
for details in shadow_details:
    cv2.circle(image_results, details['center'], 3, (0, 0, 255), -1) # Red dot at center
    cv2.putText(image_results, f"Shadow (A={details['area']:.0f})",
                (details['center'][0] + 10, details['center'][1]),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

axes3[2].imshow(cv2.cvtColor(image_results, cv2.COLOR_BGR2RGB)) # Convert BGR (OpenCV) to RGB (Matplotlib)
axes3[2].set_title('Detected Features Overlay')
axes3[2].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

print("\n--- Analysis Summary ---")
print(f"Disk Center: ({center_x}, {center_y}), Radius: {radius}")
print(f"Number of potential Zones (intensity peaks): {len(peaks)}")
print(f"Number of potential Belts (intensity troughs): {len(troughs)}")
if len(dominant_periods) > 0:
    print(f"Dominant band spatial periods from FFT: {np.round(dominant_periods, 1)} pixels")
else:
    print("No clear dominant frequency found for bands in FFT.")
print(f"Number of detected moon shadow candidates: {len(detected_shadows)}")
if len(shadow_details) > 0:
    print(f"Details of first shadow: Center={shadow_details[0]['center']}, Area={shadow_details[0]['area']:.1f}, Solidity={shadow_details[0]['solidity']:.2f}")
print("--- End of Analysis ---")

plt.show() # Display all figures

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks # For identifying bands/zones automatically

# --- Configuration ---
REFERENCE_MAP_PATH = 'T2.jpg' # Make sure T2.jpg is accessible

# --- Load the Reference Map ---
if not os.path.exists(REFERENCE_MAP_PATH):
    print(f"Error: Reference map '{REFERENCE_MAP_PATH}' not found.")
    # Exit or handle error appropriately
else:
    print(f"Loading reference map: {REFERENCE_MAP_PATH}")
    reference_image_color = cv2.imread(REFERENCE_MAP_PATH, cv2.IMREAD_COLOR)

    if reference_image_color is None:
        print(f"Error: Failed to load reference map image: {REFERENCE_MAP_PATH}")
    else:
        # --- 1. Convert to Grayscale ---
        reference_image_gray = cv2.cvtColor(reference_image_color, cv2.COLOR_BGR2GRAY)
        height, width = reference_image_gray.shape
        print(f"Reference map dimensions: Height (latitudes)={height}, Width (longitudes)={width}")

        # --- 2. Calculate Mean Latitudinal Brightness Profile ---
        # For each row (latitude), calculate the mean brightness across all columns (longitudes)
        latitudinal_profile = np.mean(reference_image_gray, axis=1)

        # --- 3. Plot the Profile ---
        # Create a y-axis that represents latitudes (e.g., pixel row from North=0 to South=height-1)
        # Or, more abstractly, just pixel row index.
        # For Jupiter, it's common to map latitude from -90 (South) to +90 (North).
        # Assuming image top is North (+90) and bottom is South (-90) for this example mapping:
        # If your map has South at top, you'd flip this or the profile.
        # Let's use pixel row for simplicity first, then map to an arbitrary latitude scale.
        
        pixel_rows = np.arange(height)
        
        # To map pixel rows to a latitude range like -90 to 90 (assuming image top is +90, bottom is -90)
        # This depends on how T2.jpg is oriented. Typically, astronomical maps have North up.
        # If T2.jpg is like typical maps, row 0 is North Pole, row height/2 is equator, row height-1 is South Pole.
        # Let's assume row 0 = +90 deg (North), row height-1 = -90 deg (South) for plotting.
        # However, many cylindrical maps place 0 deg latitude at the center row.
        # For T2.jpg, it looks like the equator (0 deg) is at the center row.
        # So row 0 is North, row height/2 is Equator, row height-1 is South.
        # Let's make a latitude axis from +90 (top) to -90 (bottom) if North is up.
        # If your T2.jpg has south up, you might need to flip the latitudinal_profile or latitudes array.

        # Assuming North is up (row 0 = North Pole, row height-1 = South Pole for plotting example)
        # Let's plot against pixel row first for directness, then discuss latitude mapping.

        plt.figure(figsize=(10, 12))

        # Plot 1: The reference map itself
        plt.subplot(2, 1, 1) # 2 rows, 1 column, first plot
        plt.imshow(cv2.cvtColor(reference_image_color, cv2.COLOR_BGR2RGB))
        plt.title(f"Reference Map: {os.path.basename(REFERENCE_MAP_PATH)}")
        # Show y-axis as pixel row for clarity with profile plot
        plt.ylabel("Pixel Row (Approx. Latitude)")
        # Add some ticks to correspond with profile
        # Determine tick positions based on number of desired ticks
        num_y_ticks = 7 # e.g. North Pole, +60, +30, Equator, -30, -60, South Pole
        y_tick_positions = np.linspace(0, height -1 , num_y_ticks)
        y_tick_labels = np.linspace(90, -90, num_y_ticks) # Example mapping to latitude
        
        # To map more accurately if height/2 is equator (0 deg)
        # If image top is North (+90), bottom is South (-90)
        # pixel_row_to_latitude = lambda r: 90 - (r / (height -1)) * 180
        # latitudes_for_plot = pixel_row_to_latitude(pixel_rows)

        plt.yticks(y_tick_positions, [f"{val:.0f}°" for val in y_tick_labels])


        # Plot 2: The Latitudinal Profile
        plt.subplot(2, 1, 2) # 2 rows, 1 column, second plot
        plt.plot(latitudinal_profile, pixel_rows) # Plot brightness vs. pixel row
        plt.gca().invert_yaxis() # Typically, North is up, so higher pixel row is South
                                 # Or, if plotting vs latitude, +90 at top.
                                 # Let's keep pixel row 0 at top for now matching image display.

        plt.title("Mean Latitudinal Brightness Profile of Reference Map")
        plt.xlabel("Mean Pixel Intensity (0-255)")
        plt.ylabel("Pixel Row (0 = Top/North, increasing downwards)")
        plt.grid(True, linestyle=':', alpha=0.7)
        
        # Match y-ticks with image plot for easier comparison
        plt.yticks(y_tick_positions, [f"{val:.0f}° (row ~{int(pos)})" for val,pos in zip(y_tick_labels, y_tick_positions)])


        # --- 4. (Optional) Automatically find peaks (Zones) and valleys (Belts) ---
        # Zones are bright (peaks in profile)
        # Belts are dark (valleys in profile, or peaks in -profile)
        
        # Finding Zones (peaks)
        # distance: min horizontal distance between peaks
        # prominence: min vertical distance to be considered a peak
        min_peak_distance = height // 20 # Heuristic for minimum separation
        min_prominence = 5 # Heuristic for minimum brightness difference
        
        zones_indices, _ = find_peaks(latitudinal_profile, distance=min_peak_distance, prominence=min_prominence)
        
        # Finding Belts (valleys, so find peaks in the inverted profile)
        belts_indices, _ = find_peaks(-latitudinal_profile, distance=min_peak_distance, prominence=min_prominence)
        
        print(f"\nDetected Zones (Bright Bands) at approx. pixel rows: {zones_indices}")
        print(f"Detected Belts (Dark Bands) at approx. pixel rows: {belts_indices}")

        # Plot detected zones and belts on the profile
        plt.plot(latitudinal_profile[zones_indices], zones_indices, "x", color='red', label='Zones (Bright)', markersize=8)
        plt.plot(latitudinal_profile[belts_indices], belts_indices, "o", color='blue', mfc='none', label='Belts (Dark)', markersize=8)
        plt.legend()
        
        plt.tight_layout()
        plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import find_peaks

REFERENCE_MAP_PATH = 'T2.jpg'

if not os.path.exists(REFERENCE_MAP_PATH):
    print(f"Error: Reference map '{REFERENCE_MAP_PATH}' not found.")
else:
    print(f"Loading reference map: {REFERENCE_MAP_PATH}")
    reference_image_color = cv2.imread(REFERENCE_MAP_PATH, cv2.IMREAD_COLOR)

    if reference_image_color is None:
        print(f"Error: Failed to load reference map image: {REFERENCE_MAP_PATH}")
    else:
        reference_image_gray = cv2.cvtColor(reference_image_color, cv2.COLOR_BGR2GRAY)
        height, width = reference_image_gray.shape
        print(f"Reference map dimensions: Height (latitudes)={height}, Width (longitudes)={width}")

        latitudinal_profile = np.mean(reference_image_gray, axis=1)
        pixel_rows = np.arange(height)
        
        # For y-axis ticks and labels (approximate latitude mapping)
        num_y_ticks = 7 
        y_tick_positions_for_profile_plot = np.linspace(0, height -1 , num_y_ticks)
        # Assuming row 0 is North (+90), row height/2 is Equator (0), row height-1 is South (-90)
        y_tick_labels_for_profile_plot = np.linspace(90, -90, num_y_ticks)


        min_peak_distance = height // 20 
        min_prominence = 5 
        
        zones_indices, _ = find_peaks(latitudinal_profile, distance=min_peak_distance, prominence=min_prominence)
        belts_indices, _ = find_peaks(-latitudinal_profile, distance=min_peak_distance, prominence=min_prominence)
        
        print(f"\nDetected Zones (Bright Bands) at approx. pixel rows: {zones_indices}")
        print(f"Detected Belts (Dark Bands) at approx. pixel rows: {belts_indices}")

        # --- Create 3 Subplots ---
        fig, axes = plt.subplots(1, 3, figsize=(24, 8)) # 1 row, 3 columns

        # Plot 1: The reference map itself
        axes[0].imshow(cv2.cvtColor(reference_image_color, cv2.COLOR_BGR2RGB))
        axes[0].set_title(f"1. Reference Map: {os.path.basename(REFERENCE_MAP_PATH)}")
        axes[0].set_ylabel("Pixel Row (Approx. Latitude)")
        axes[0].set_yticks(y_tick_positions_for_profile_plot)
        axes[0].set_yticklabels([f"{val:.0f}°" for val in y_tick_labels_for_profile_plot])

        # Plot 2: The Latitudinal Profile (Separate Plot)
        axes[1].plot(latitudinal_profile, pixel_rows) 
        axes[1].invert_yaxis() 
        axes[1].set_title("2. Mean Latitudinal Brightness Profile")
        axes[1].set_xlabel("Mean Pixel Intensity (0-255)")
        axes[1].set_ylabel("Pixel Row / Approx. Latitude")
        axes[1].grid(True, linestyle=':', alpha=0.7)
        axes[1].set_yticks(y_tick_positions_for_profile_plot)
        axes[1].set_yticklabels([f"{val:.0f}° (row ~{int(pos)})" for val,pos in zip(y_tick_labels_for_profile_plot, y_tick_positions_for_profile_plot)])
        axes[1].plot(latitudinal_profile[zones_indices], zones_indices, "x", color='red', label='Zones (Bright)', markersize=8)
        axes[1].plot(latitudinal_profile[belts_indices], belts_indices, "o", color='blue', mfc='none', label='Belts (Dark)', markersize=8)
        axes[1].legend()
        
        # Plot 3: Reference Map with Profile Overlay
        axes[2].imshow(cv2.cvtColor(reference_image_color, cv2.COLOR_BGR2RGB))
        axes[2].set_title(f"3. Map with Profile Overlay")
        
        # Overlay the profile line
        # The profile brightness (0-255) will be plotted as x-coordinates on the image
        # pixel_rows (0 to height-1) will be the y-coordinates
        axes[2].plot(latitudinal_profile, pixel_rows, color='yellow', linewidth=1.5, alpha=0.8)
        
        # Overlay detected zones and belts markers on the image
        # For x-coordinate of markers, we use the profile value at that zone/belt index.
        # For y-coordinate, we use the zone/belt index itself (pixel row).
        axes[2].scatter(latitudinal_profile[zones_indices], zones_indices, 
                        marker='x', color='red', s=50, label='Zones (Overlay)', alpha=0.9,linewidths=1.5)
        axes[2].scatter(latitudinal_profile[belts_indices], belts_indices, 
                        marker='o', edgecolor='cyan', facecolor='none', s=50, label='Belts (Overlay)', alpha=0.9, linewidths=1.5)
        
        axes[2].set_ylabel("Pixel Row (Approx. Latitude)")
        axes[2].set_xlabel("Pixel Column (Longitude / Profile Intensity Overlay)")
        axes[2].set_yticks(y_tick_positions_for_profile_plot)
        axes[2].set_yticklabels([f"{val:.0f}°" for val in y_tick_labels_for_profile_plot])
        # Optionally limit x-axis for the overlay if profile is too wide, or scale profile
        # axes[2].set_xlim(0, width) # Ensure x-axis matches image width
        axes[2].legend(fontsize='small', loc='lower right')

        plt.tight_layout()
        plt.show()
        